<p align="center">
  <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://raw.githubusercontent.com/simvue-io/.github/refs/heads/main/simvue-white.png" />
    <source media="(prefers-color-scheme: light)" srcset="https://raw.githubusercontent.com/simvue-io/.github/refs/heads/main/simvue-black.png" />
    <img alt="Simvue" src="https://github.com/simvue-io/.github/blob/5eb8cfd2edd3269259eccd508029f269d993282f/simvue-black.png" width="500">
  </picture>
</p>

# Detailed Example using Simpy
This is a more detailed example of using Simvue to track and monitor a simulation. In this case we are going to use a package called Simpy to models a bank counter and customers arriving at random times. Each customer has a certain patience. They wait to get to the counter until they're at the end of their tether. If they get to the counter, they uses it for a while before releasing it for the next customer to use.

This is based on the Bank Renege example from the Simpy documentation - [see the full example here](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html)


### Install dependencies
Install any dependencies if you have not already done so:

In [ ]:
!pip install simvue simpy numpy


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip



### Initialisation
To proceed you need to specify the URL of the Simvue server and provide an access token used to authenticate to the server. This can be done by either creating a `simvue.toml` file containing the required details, or specifying them as environment variables.

Login to https://app.simvue.io, go to the **Runs** page and click **Create new run**. Copy the 'token' from here. The run the cell below, paste the token into the box when prompted and push enter.

In [7]:
import os
import getpass

os.environ["SIMVUE_URL"] = "https://nightly.simvue.io"
os.environ["SIMVUE_TOKEN"] = getpass.getpass(prompt="Token: ")

### Creating the Model
Now we are going to create our simulation of the bank. Firstly, we will import our required modules and define some constants which will be used throughout the simulation:

In [11]:
import random
import simpy
import simvue
import numpy
import time

RANDOM_SEED = 42 # This makes the simulation reproducible - change it to get a new, randomised simulation
NEW_CUSTOMERS = 40  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Minimum customer patience (seconds)
MAX_PATIENCE = 5  # Maximmum customer patience (seconds)

We then create a function which defines the behaviour of each customer, passing in the following parameters:
* **env**: The simulation environment.
* **name**: The customer’s name.
* **counter**: The resource representing the bank counter.
* **time_in_bank**: Average time a customer spends at the counter.
* **run**: The Simvue Run object for tracking the simulation

In [4]:
def customer(env, name, counter, time_in_bank, run):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    # Log an event with Simvue for when each customer arrives at the bank
    run.log_event(f'{arrive:7.4f} {name}: Here I am!')

    # The customer requests to access the counter
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        
        # Wait for the counter to become available, or abort once the customer has ran out of patience
        results = yield req | env.timeout(patience)

        # Record how long they waited at the counter
        wait = env.now - arrive

        if req in results:
            # The customer got to the counter
            
            # Log an event to show that they have been served
            run.log_event(f'{env.now:7.4f} {name}: SERVED after {wait:6.3f}')
            
            # The customer then spends a random amount of time at the counter (exponential distribution around the average time we specified)
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            
            # Log an event once they have finished being served
            run.log_event(f'{env.now:7.4f} {name}: Finished')

        else:
            # The customer gave up - increment counter and log an event
            env.reneged_customers += 1
            run.log_event(f'{env.now:7.4f} {name}: RENEGED after {wait:6.3f}')

        # Update statistics - record wait time, average wait time for all customers, and percentage who reneged
        env.wait_times = numpy.append(env.wait_times, wait)
        _average_wait = numpy.mean(env.wait_times)
        _percentage_reneged = env.reneged_customers / env.total_customers * 100
        
        # Log these statistics as metrics to Simvue
        run.log_metrics({"percentage_reneged": _percentage_reneged, "average_wait": _average_wait}, time=env.now)

We then define a source function - this is used to generate our bank customers at semi random intervals. We pass in the following variables to this function:
* **env**: The simulation environment.
* **number**: Number of customers to generate.
* **interval**: Average interval between customer arrivals.
* **counter**: The resource representing the bank counter.
* **run**: The Simvue Run object for tracking the simulation

In [5]:
def source(env, number, interval, counter, run):
    """Source generates customers randomly"""
    # Generate a new customer, process it, and then wait for a random length of time before creating another one
    for i in range(number):
        env.total_customers += 1
        c = customer(env, f'Customer{i:02d}', counter, time_in_bank=12.0, run=run)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

Next we want to set up our Simvue run and start the simulation. To do this we use the `Run` class from Simvue as a context manager, and call the `init` method. We then add any additional information we want to store, before running the simulation:

In [ ]:
# Setup the simulation (will run it in real time)
random.seed(RANDOM_SEED)
env = simpy.rt.RealtimeEnvironment(factor=1.0, strict=False)

# Initialize statisticss as part of the env object
env.total_customers = 0
env.reneged_customers = 0
env.wait_times = numpy.array([])

# Start Simvue run as a context manager and initialize the run
with simvue.Run() as run:
    run.init(
        name="bank-customers-example-%d" % time.time(),
        folder="/examples",
        description="Simulate customers being served at a bank, recording the wait times and percentage who don't get served.",
        tags=["example", "bank-customers"],
        notification="all"
    )
    
    # Upload metadata which corresponds to the variables we defined at the beginning
    run.update_metadata(
        {
            "random_seed": RANDOM_SEED,
            "num_customers": NEW_CUSTOMERS,
            "average_customer_interval": INTERVAL_CUSTOMERS,
            "customer_min_patience": MIN_PATIENCE,
            "customer_max_patience": MAX_PATIENCE
        }
    )
    
    # Upload this file as a code artifact
    run.save_file(os.path.join(os.getcwd(), "simvue_detailed_example.ipynb"), category="code")
    
    # Add some alerts so that we can be notified if things go wrong
    
    # For example, could add an Event based alert which is triggered when a customer gives up
    run.create_event_alert(
        name="customer_reneged",
        pattern="RENEGED",
        description="A bank customer gave up before being served!"
    )
    # Or a Metric based alert which is triggered when the percentage reneged is above 40%
    run.create_metric_threshold_alert(
        name="customer_reneged_above_40_percent",
        metric="percentage_reneged",
        threshold=40,
        rule="is above",
        description="More than 40 percent of customers are giving up before being served!",
        notification="email",
        window=1
    )
    
    # Start processes and run the simulation
    counter = simpy.Resource(env, capacity=1)
    env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, run))
    env.run()
    
    # Once simulation is complete, save our array of wait times as an output Artifact
    run.save_object(env.wait_times, category='output', name='wait_times')
    
    # Let's say if more than 50% of customers gave up before being served, our run is a failure
    if (env.reneged_customers / env.total_customers) > 0.5:
        run.set_status("failed")


[simvue] Run bank-customers-example-1740158722 created
[simvue] Monitor in the UI at https://nightly.simvue.io/dashboard/runs/run/4YqZawDmUCTYYvXAzE6QQ5


### Results
That's it! You can now view the run in the Simvue UI by clicking on the link above. You should be able to see:
* A new run has been created in the `/examples` folder, with the name, tags and description which we specified in the `init` method
* The run has a set of metadata detailing the variables we used in our simulation, along with some automatically collected information like the Python environment used
* This notebook has been uploaded as a Code artifact, and once the simulation has finished our array of wait times is uploaded as an Output artifact
* There are two metrics, `average_wait` and `percentage_reneged`, which are updating live as the simulation progresses
* The events log shows each customer arriving, waiting, and either being served or reneging
* There are two alerts:
    - One based on the events log, which should fire near the start of the run when the first customer gives up without being served
    - One based on the `percentage_reneged` metric, which fires near the end of the simulation when the percentage of customers who reneged reached 40%. This one should also send you an email
* The run's status is set to Failed as the final percentage of customers giving up exceeds 50%
* You received an email when the run finished, telling you that it failed

Try tweaking the input parameters, and see what effect it has! Compare different runs easily using the Simvue web UI, with the ability to filter based on things like run status, tags, and metadata to identify the runs you care about. and creating custom plots to visualise your results.

(If you want to make the simulation run more quickly to experiment, reduce the `factor` parameter)